In [21]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

In [22]:
df = pd.read_csv('TCS.csv')
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2003-01-01,59.987499,61.974998,59.987499,61.087502,45.255402,6027072.0
1,2003-01-02,61.525002,62.924999,57.912498,58.299999,43.190327,5325328.0
2,2003-01-03,60.000000,61.049999,58.500000,59.012501,43.718178,4198040.0
3,2003-01-06,59.924999,60.187500,56.875000,57.262501,42.421741,4121520.0
4,2003-01-07,58.000000,58.500000,56.062500,56.599998,41.930927,2650800.0
...,...,...,...,...,...,...,...
4719,2021-12-24,3685.000000,3705.000000,3644.800049,3670.899902,3664.460693,2209923.0
4720,2021-12-27,3671.000000,3700.000000,3653.100098,3696.100098,3689.616699,1534135.0
4721,2021-12-28,3710.000000,3725.000000,3693.850098,3706.550049,3700.048340,1456218.0
4722,2021-12-29,3692.250000,3719.949951,3685.000000,3694.699951,3688.218994,1456923.0


In [23]:
df = df[['Close']]
df.dropna(inplace = True)
df.head()

c:\users\mahen\appdata\local\programs\python\python39\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,Close
0,61.087502
1,58.299999
2,59.012501
3,57.262501
4,56.599998


In [24]:
scaler = MinMaxScaler(feature_range=(0,1))
data = scaler.fit_transform(df)

In [25]:
trainSize = int(len(data)*0.8)
testSize = len(data) - trainSize
train = data[0:trainSize]
test = data[trainSize:len(data)]

In [26]:
print(data.shape)
print(train.shape)
print(test.shape)

(4715, 1)
(3772, 1)
(943, 1)


In [27]:
def modify(data, pastDays = 14):
  X = []
  Y = []
  for i in range(pastDays, len(data)):
      X.append(data[i - pastDays: i, 0: data.shape[1]])
      Y.append(data[i : i + 1, 0])
  return np.array(X), np.array(Y)

In [28]:
trainX, trainY = modify(train)
testX, testY = modify(test)

RNN

In [29]:
model1 = Sequential()
model1.add(InputLayer((trainX.shape[1], trainX.shape[2])))
model1.add(SimpleRNN(150, return_sequences = True))
model1.add(SimpleRNN(150, return_sequences = True))
model1.add(SimpleRNN(150))
model1.add(Dense(trainY.shape[1]))
model1.compile(optimizer='adam', loss='mse')
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_3 (SimpleRNN)    (None, 14, 150)           22800     
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 14, 150)           45150     
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 150)               45150     
                                                                 
 dense_1 (Dense)             (None, 1)                 151       
                                                                 
Total params: 113,251
Trainable params: 113,251
Non-trainable params: 0
_________________________________________________________________


In [30]:
model1.fit(trainX, trainY, validation_split = 0.2, epochs = 50)

Epoch 1/50
94/94 [==============================] - 5s 25ms/step - loss: 0.0720 - val_loss: 1.5704e-04
Epoch 2/50
94/94 [==============================] - 2s 24ms/step - loss: 4.5843e-05 - val_loss: 1.6273e-04
Epoch 3/50
94/94 [==============================] - 2s 20ms/step - loss: 4.2205e-05 - val_loss: 1.5780e-04
Epoch 4/50
94/94 [==============================] - 2s 22ms/step - loss: 3.5917e-05 - val_loss: 1.1366e-04
Epoch 5/50
94/94 [==============================] - 2s 18ms/step - loss: 4.4693e-05 - val_loss: 1.1324e-04
Epoch 6/50
94/94 [==============================] - 2s 16ms/step - loss: 4.4124e-05 - val_loss: 2.0435e-04
Epoch 7/50
94/94 [==============================] - 2s 16ms/step - loss: 4.7158e-05 - val_loss: 2.4778e-04
Epoch 8/50
94/94 [==============================] - 2s 17ms/step - loss: 6.0694e-05 - val_loss: 9.9766e-05
Epoch 9/50
94/94 [==============================] - 2s 16ms/step - loss: 1.0355e-04 - val_loss: 3.4703e-04
Epoch 10/50
94/94 [======================

In [31]:
model1.save('RNN(Uni)')

INFO:tensorflow:Assets written to: RNN(Uni)\assets


Classic LSTM

In [32]:
model2 = Sequential()
model2.add(InputLayer((trainX.shape[1], trainX.shape[2])))
model2.add(LSTM(150, return_sequences = True))
model2.add(LSTM(150, return_sequences = True))
model2.add(LSTM(150))
model2.add(Dense(trainY.shape[1]))
model2.compile(optimizer='adam', loss='mse')
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 14, 150)           91200     
                                                                 
 lstm_1 (LSTM)               (None, 14, 150)           180600    
                                                                 
 lstm_2 (LSTM)               (None, 150)               180600    
                                                                 
 dense_2 (Dense)             (None, 1)                 151       
                                                                 
Total params: 452,551
Trainable params: 452,551
Non-trainable params: 0
_________________________________________________________________


In [33]:
model2.fit(trainX, trainY, validation_split = 0.2, epochs = 50)

Epoch 1/50
94/94 [==============================] - 12s 69ms/step - loss: 6.7417e-04 - val_loss: 1.2544e-04
Epoch 2/50
94/94 [==============================] - 4s 47ms/step - loss: 3.4924e-05 - val_loss: 1.3224e-04
Epoch 3/50
94/94 [==============================] - 4s 47ms/step - loss: 3.7684e-05 - val_loss: 2.4553e-04
Epoch 4/50
94/94 [==============================] - 4s 44ms/step - loss: 4.8706e-05 - val_loss: 1.3207e-04
Epoch 5/50
94/94 [==============================] - 4s 44ms/step - loss: 4.0693e-05 - val_loss: 1.3885e-04
Epoch 6/50
94/94 [==============================] - 6s 61ms/step - loss: 3.5957e-05 - val_loss: 1.8580e-04
Epoch 7/50
94/94 [==============================] - 4s 45ms/step - loss: 4.1352e-05 - val_loss: 1.0371e-04
Epoch 8/50
94/94 [==============================] - 4s 45ms/step - loss: 3.3140e-05 - val_loss: 1.2840e-04
Epoch 9/50
94/94 [==============================] - 6s 61ms/step - loss: 3.9449e-05 - val_loss: 1.4874e-04
Epoch 10/50
94/94 [=================

In [34]:
model2.save('LSTM(Uni)')

INFO:tensorflow:Assets written to: LSTM(Uni)\assets


INFO:tensorflow:Assets written to: LSTM(Uni)\assets


LSTM Peephole

In [35]:
model3 = Sequential()
model3.add(InputLayer((trainX.shape[1], trainX.shape[2])))
model3.add(RNN(tfa.rnn.PeepholeLSTMCell(150), return_sequences = True))
model3.add(RNN(tfa.rnn.PeepholeLSTMCell(150), return_sequences = True))
model3.add(RNN(tfa.rnn.PeepholeLSTMCell(150)))
model3.add(Dense(trainY.shape[1]))
model3.compile(optimizer='adam', loss='mse')
model3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rnn (RNN)                   (None, 14, 150)           91650     
                                                                 
 rnn_1 (RNN)                 (None, 14, 150)           181050    
                                                                 
 rnn_2 (RNN)                 (None, 150)               181050    
                                                                 
 dense_3 (Dense)             (None, 1)                 151       
                                                                 
Total params: 453,901
Trainable params: 453,901
Non-trainable params: 0
_________________________________________________________________


In [36]:
model3.fit(trainX, trainY, validation_split = 0.2, epochs = 50)

Epoch 1/50
94/94 [==============================] - 12s 68ms/step - loss: 6.2772e-04 - val_loss: 1.1594e-04
Epoch 2/50
94/94 [==============================] - 5s 49ms/step - loss: 3.4344e-05 - val_loss: 1.5662e-04
Epoch 3/50
94/94 [==============================] - 5s 51ms/step - loss: 4.3497e-05 - val_loss: 1.1775e-04
Epoch 4/50
94/94 [==============================] - 4s 48ms/step - loss: 4.4075e-05 - val_loss: 1.3527e-04
Epoch 5/50
94/94 [==============================] - 4s 47ms/step - loss: 4.1705e-05 - val_loss: 1.1544e-04
Epoch 6/50
94/94 [==============================] - 4s 46ms/step - loss: 6.2239e-05 - val_loss: 1.0244e-04
Epoch 7/50
94/94 [==============================] - 5s 53ms/step - loss: 3.2944e-05 - val_loss: 1.2858e-04
Epoch 8/50
94/94 [==============================] - 6s 65ms/step - loss: 3.1856e-05 - val_loss: 1.1204e-04
Epoch 9/50
94/94 [==============================] - 5s 51ms/step - loss: 3.7187e-05 - val_loss: 2.3976e-04
Epoch 10/50
94/94 [=================

In [37]:
model3.save('PeepholeLSTM(Uni)')

INFO:tensorflow:Assets written to: PeepholeLSTM(Uni)\assets


INFO:tensorflow:Assets written to: PeepholeLSTM(Uni)\assets


Bidirectional LSTM

In [38]:
model4 = Sequential()
model4.add(InputLayer((trainX.shape[1], trainX.shape[2])))
model4.add(Bidirectional(LSTM(150, return_sequences = True)))
model4.add(Bidirectional(LSTM(150, return_sequences = True)))
model4.add(Bidirectional(LSTM(150)))
model4.add(Dense(trainY.shape[1]))
model4.compile(optimizer='adam', loss='mse')
model4.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 14, 300)          182400    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 14, 300)          541200    
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 300)              541200    
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 1)                 301       
                                                                 
Total params: 1,265,101
Trainable params: 1,265,101
Non-trainable params: 0
____________________________________________

In [39]:
model4.fit(trainX, trainY, validation_split = 0.2, epochs = 50)

Epoch 1/50
94/94 [==============================] - 48s 204ms/step - loss: 5.0032e-04 - val_loss: 1.0509e-04
Epoch 2/50
94/94 [==============================] - 13s 141ms/step - loss: 3.6427e-05 - val_loss: 1.6621e-04
Epoch 3/50
94/94 [==============================] - 18s 193ms/step - loss: 3.4405e-05 - val_loss: 9.3652e-05
Epoch 4/50
94/94 [==============================] - 23s 241ms/step - loss: 2.9404e-05 - val_loss: 8.8231e-05
Epoch 5/50
94/94 [==============================] - 21s 225ms/step - loss: 3.6602e-05 - val_loss: 9.2079e-05
Epoch 6/50
94/94 [==============================] - 21s 220ms/step - loss: 2.8444e-05 - val_loss: 8.8572e-05
Epoch 7/50
94/94 [==============================] - 21s 224ms/step - loss: 2.4864e-05 - val_loss: 8.0545e-05
Epoch 8/50
94/94 [==============================] - 22s 239ms/step - loss: 3.1180e-05 - val_loss: 9.7799e-05
Epoch 9/50
94/94 [==============================] - 21s 224ms/step - loss: 2.5657e-05 - val_loss: 8.8914e-05
Epoch 10/50
94/94 [

In [40]:
model4.save('BiLSTM(Uni)')

INFO:tensorflow:Assets written to: BiLSTM(Uni)\assets


INFO:tensorflow:Assets written to: BiLSTM(Uni)\assets


GRU(Gated Recurrent Unit)

In [41]:
model5 = Sequential()
model5.add(InputLayer((trainX.shape[1], trainX.shape[2])))
model5.add(GRU(150, return_sequences = True))
model5.add(GRU(150, return_sequences = True))
model5.add(GRU(150))
model5.add(Dense(trainY.shape[1]))
model5.compile(optimizer='adam', loss='mse')
model5.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 14, 150)           68850     
                                                                 
 gru_1 (GRU)                 (None, 14, 150)           135900    
                                                                 
 gru_2 (GRU)                 (None, 150)               135900    
                                                                 
 dense_5 (Dense)             (None, 1)                 151       
                                                                 
Total params: 340,801
Trainable params: 340,801
Non-trainable params: 0
_________________________________________________________________


In [42]:
model5.fit(trainX, trainY, validation_split = 0.2, epochs = 50)

Epoch 1/50
94/94 [==============================] - 48s 306ms/step - loss: 6.6225e-04 - val_loss: 5.7600e-05
Epoch 2/50
94/94 [==============================] - 10s 106ms/step - loss: 1.7115e-05 - val_loss: 9.1874e-05
Epoch 3/50
94/94 [==============================] - 10s 105ms/step - loss: 1.9385e-05 - val_loss: 6.7012e-05
Epoch 4/50
94/94 [==============================] - 10s 109ms/step - loss: 1.9051e-05 - val_loss: 5.3888e-05
Epoch 5/50
94/94 [==============================] - 10s 110ms/step - loss: 1.7282e-05 - val_loss: 6.5343e-05
Epoch 6/50
94/94 [==============================] - 11s 112ms/step - loss: 1.7989e-05 - val_loss: 1.2934e-04
Epoch 7/50
94/94 [==============================] - 11s 116ms/step - loss: 1.7287e-05 - val_loss: 4.6388e-05
Epoch 8/50
94/94 [==============================] - 12s 125ms/step - loss: 1.9827e-05 - val_loss: 1.2109e-04
Epoch 9/50
94/94 [==============================] - 11s 117ms/step - loss: 2.0317e-05 - val_loss: 5.0569e-05
Epoch 10/50
94/94 [

In [43]:
model5.save('GRU(Uni)')

INFO:tensorflow:Assets written to: GRU(Uni)\assets


INFO:tensorflow:Assets written to: GRU(Uni)\assets


Bidirectional GRU

In [44]:
model6 = Sequential()
model6.add(InputLayer((trainX.shape[1], trainX.shape[2])))
model6.add(Bidirectional(GRU(150, return_sequences = True)))
model6.add(Bidirectional(GRU(150, return_sequences = True)))
model6.add(Bidirectional(GRU(150)))
model6.add(Dense(trainY.shape[1]))
model6.compile(optimizer='adam', loss='mse')
model6.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_3 (Bidirectio  (None, 14, 300)          137700    
 nal)                                                            
                                                                 
 bidirectional_4 (Bidirectio  (None, 14, 300)          406800    
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 300)              406800    
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 1)                 301       
                                                                 
Total params: 951,601
Trainable params: 951,601
Non-trainable params: 0
________________________________________________

In [45]:
model6.fit(trainX, trainY, validation_split = 0.2, epochs = 50)

Epoch 1/50
94/94 [==============================] - 99s 204ms/step - loss: 3.7446e-04 - val_loss: 8.1190e-05
Epoch 2/50
94/94 [==============================] - 17s 176ms/step - loss: 2.2620e-05 - val_loss: 6.6592e-05
Epoch 3/50
94/94 [==============================] - 17s 179ms/step - loss: 2.2878e-05 - val_loss: 6.8621e-05
Epoch 4/50
94/94 [==============================] - 16s 168ms/step - loss: 2.0687e-05 - val_loss: 1.7325e-04
Epoch 5/50
94/94 [==============================] - 15s 159ms/step - loss: 1.9434e-05 - val_loss: 1.3009e-04
Epoch 6/50
94/94 [==============================] - 15s 161ms/step - loss: 1.6151e-05 - val_loss: 4.1687e-05
Epoch 7/50
94/94 [==============================] - 15s 163ms/step - loss: 1.5943e-05 - val_loss: 4.7104e-05
Epoch 8/50
94/94 [==============================] - 16s 165ms/step - loss: 1.5235e-05 - val_loss: 6.6582e-05
Epoch 9/50
94/94 [==============================] - 14s 154ms/step - loss: 1.7269e-05 - val_loss: 5.7375e-05
Epoch 10/50
94/94 [

In [46]:
model6.save('BiGRU(Uni)')

INFO:tensorflow:Assets written to: BiGRU(Uni)\assets


INFO:tensorflow:Assets written to: BiGRU(Uni)\assets
